In [11]:
import os

In [12]:
import json
import pandas as pd
import traceback

In [13]:
from langchain.chat_models import ChatOpenAI

In [16]:
from dotenv import load_dotenv

load_dotenv()

True

In [17]:
KEY = os.getenv("OPENAI_API_KEY")

In [18]:
print(KEY)

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
